# Gap creation rate
Inspired from Carlberg et al 2012 

$$ R(r) = \int _M \int _{v_\perp} \int_{0}^{b_{\textrm{max}}} n(r) N(M) v_\perp f(v_\perp) \pi db dv_\perp dM$$ 

for me, I am doing something numerical! so my result is:

$$ R_{\textrm{Pal 5}} =  \frac{1}{T}\int_{0}^T \mathcal{l}^{-1}(t) \sum_i \delta(t-t_i) dt$$

The delta function is zero everywhere except at $t_i$ which is the time at which the impacts occur. $\mathcal{l}$ is the length of the stream, and $T$ is the total integration time. Therefore, for a given simulation:

$$ R_{j,\textrm{Pal 5}} =  \frac{1}{T} \sum_i \frac{1}{l (t_i)}$$


And then we look at the distribution of R for all the simulations: 

$$ \langle R_{\textrm{Pal 5}}\rangle = \textrm{Mean}(R_{j,\textrm{Pal 5}}) $$

In [1]:
import gcs


In [3]:
# must now get the tauDensityMaps from the path_handler to find the lengthat each moment in time
gcs.path_handler.tauDensityMaps?

Signature:
gcs.path_handler.tauDensityMaps(
    GCname,
    MWpotential,
    montecarlokey,
    NP,
    internal_dynamics,
)
Docstring: <no docstring>
File:      ~/miniconda3/envs/stream_analysis/lib/python3.11/site-packages/gcs-0.1-py3.11.egg/gcs/path_handler.py
Type:      function